In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [2]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
import LinearAlgebra: ldiv!, dot
using Test
using ForwardDiff
using SpecialFunctions
using BenchmarkTools
using TransportMap
using QuadGK
using Polynomials
using Distributions
using Random
using LoopVectorization
using Optim
using NLsolve
using MLDataUtils
using MLDataPattern
using Test
using SparseArrays
using AdaptiveTransportMap: vander, transform!, evaluate, ncoeff, optimize, negative_log_likelihood!, derivative

# using Profile
# using ProfileView
using StaticArrays
using QRupdate

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1278


In [8]:
Nx = 2
Ne = 8
m = 20

X =  Matrix([0.267333   1.43021;
      0.364979   0.607224;
     -1.23693    0.249277;
     -2.0526     0.915629;
     -0.182465   0.415874;
      0.412907   1.01672;
      1.41332   -0.918205;
      0.766647  -1.00445]');

# X = randn(Nx, Ne)

idx = [0 0; 0 1; 1 0]

Nψ = 3#size(idx,1)

coeff = [-0.28829821609289064;
         -1.1768877197305714;
          0.40090743090076547];
#           1.101330816362896];
C = MapComponent(m, Nx, idx, coeff; α = 0.0);

In [11]:
# R = UpperTriangular(qr(S.ψoff .* S.ψd).R)
@time S = Storage(C.I.f, X)
# F = QRscaling(R, S.ψnorm);

  0.000020 seconds (32 allocations: 5.281 KiB)


In [48]:
c̃oeff = F.U*coeff

3-element Array{Float64,1}:
 4.05746087637728
 0.3881750820002722
 0.2304407164032014

In [50]:
lmul!(F.Uinv, c̃oeff)

3-element Array{Float64,1}:
 -0.28829821609289064
 -1.1768877197305716
  0.4009074309007655

In [15]:
J = 0.0
dJ = zeros(Nψ)
@time negative_log_likelihood!(J, dJ, coeff, S, C, X)

  0.000042 seconds (153 allocations: 16.922 KiB)


1.3271538795196385

In [16]:
dJ

3-element Array{Float64,1}:
 -0.4251153538710222
 -0.22136527273698292
 -0.18485216244465474

In [49]:
J = 0.0
dJ = zeros(Nψ)
negative_log_likelihood!(J, dJ, coeff, S, C, X)

1.3271538795196385

In [42]:
J̃ = 0.0
dJ̃ = zeros(Nψ)
qrnegative_log_likelihood!(J̃, dJ̃, c̃oeff, D, R, Rinv, S, C, X)

LoadError: DomainError with 0.5:
integrand produced NaN in the interval (0.0, 1.0)

### With update of the storage

In [ ]:
Nx = 2
Ne = 8
m = 20

X =  Matrix([0.267333   1.43021;
      0.364979   0.607224;
     -1.23693    0.249277;
     -2.0526     0.915629;
     -0.182465   0.415874;
      0.412907   1.01672;
      1.41332   -0.918205;
      0.766647  -1.00445]');

# X = randn(Nx, Ne)

idx = [0 0; 0 1; 1 0]

Nψ = 4#size(idx,1)

coeff = [-0.28829821609289064;
         -1.1768877197305714;
          0.40090743090076547];
#           1.101330816362896];
C = MapComponent(m, Nx, idx, coeff);

addedidx = reshape([1 1], (1,2))
addedcoeff = [0.1910678298362849]

In [6]:
fnew = ParametricFunction(ExpandedFunction(C.I.f.f.B, vcat(getidx(C), addedidx), vcat(getcoeff(C), addedcoeff)))

ParametricFunction(ExpandedFunction(20, 4, 2, MultiBasis(Basis of 20 functions: Constant -> 18th degree Probabilistic Hermite function
, 2), [0 0; 0 1; 1 0; 1 1], [-0.28829821609289064, -1.1768877197305714, 0.40090743090076547, 0.1910678298362849]))

In [7]:
ψ = evaluate_basis(fnew.f, X, getidx(C))
addedψ = evaluate_basis(fnew.f, X, addedidx)
ψnew = evaluate_basis(fnew.f, X);

F = qr(ψ);
R = F.R


8×4 Array{Float64,2}:
 1.0  0.378763  0.620434  0.234997
 1.0  0.575999  0.610931  0.351895
 1.0  0.621883  0.430862  0.267945
 1.0  0.512188  0.220301  0.112835
 1.0  0.604891  0.626383  0.378894
 1.0  0.487776  0.605263  0.295233
 1.0  0.511584  0.383338  0.19611
 1.0  0.490809  0.545307  0.267642